In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely import wkt
from folium import FeatureGroup, LayerControl, Map, Marker
from folium.plugins import HeatMap
import folium
import branca
import branca.colormap as cm


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('../input/newyork-subway-entry-exit-dataset-for-dataviz/rows.csv')
data.T.head(50)

In [ ]:
data.dtypes

In [ ]:
data.columns

In [ ]:
plt.figure(figsize=(20,20))
sns.scatterplot(x='Station Longitude', y='Station Latitude', hue='Division',s=20, data=data)

In [ ]:
plt.figure(figsize=(20,20))
sns.scatterplot(x='Station Longitude', y='Station Latitude', hue='Line',s=20, data=data)

In [ ]:
import matplotlib.pyplot as plt



#sns.scatterplot(x='Station Longitude', y='Station Latitude', hue='Line',s=20, data=data)

data.plot(kind="scatter", x="Station Longitude", y="Station Latitude",
    #s=data['Line'].count(), 
    label="Line",
    #cmap=plt.get_cmap("jet"),
    colorbar=True, alpha=0.4, figsize=(10,7),
)
plt.legend()
plt.show()

In [ ]:
div_count = data.groupby('Division').agg('count').reset_index()
line_count = data.groupby('Line').agg('count').reset_index()
station_count = data.groupby('Station Name').agg('count').reset_index()

print(div_count.head())
print(line_count.head())
print(station_count.head())


In [ ]:
fig, ax1 = plt.subplots(1,1, figsize=(10,10))
sns.barplot(x='Division', y='Line', data=div_count)
ax1.set_title('Number of Entries by Division', fontsize=15)
ax1.set_xlabel('Division', fontsize=12)
ax1.set_ylabel('Count', fontsize=12)

fig, ax2 = plt.subplots(1,1, figsize=(10,10))
sns.barplot(y='Line', x='Division', data=line_count)
ax2.set_title('Number of Entries by Line', fontsize=15)
ax2.set_xlabel('Line', fontsize=12)
ax2.set_ylabel('Count', fontsize=12)

fig, ax3 = plt.subplots(1,1, figsize=(10,100))
sns.barplot(x='Line', y='Station Name', data=station_count, order=station_count.sort_values('Line', ascending = False)["Station Name"])
ax3.set_title('Number of Entries by Station', fontsize=15)
ax3.set_xlabel('Station', fontsize=12)
ax3.set_ylabel('Count', fontsize=12)

In [ ]:
nyc = gpd.read_file(gpd.datasets.get_path('nybb'))
nyc.head(5)

In [ ]:
train_routes = data.groupby(['Station Latitude','Station Longitude','Station Name'])['Line'].count().reset_index() #.rename(columns={'id':'Num_Trips'})
print(train_routes.head())
print(train_routes.Line.unique())

In [ ]:
colors=cm.linear.Paired_12.scale(1, 12)
train_routes_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 10.5,)
for i in range(0,len(train_routes)):
    folium.Circle(
      location=[train_routes.iloc[i]['Station Latitude'], train_routes.iloc[i]['Station Longitude']],
        radius=3,
        fill=True,
        color=colors(train_routes.iloc[i]['Line']),
        popup=train_routes.iloc[i]['Station Name'],
        tooltip=train_routes.iloc[i]['Station Name'],
    ).add_to(train_routes_map)
train_routes_map

In [ ]:
data.describe()

In [ ]:
#data=data.replace(to_replace = np.nan, value = -99) 

In [ ]:
plt.figure(figsize=(10,8), dpi= 80)
sns.pairplot(data, kind="scatter", hue="Line", plot_kws=dict(s=80, edgecolor="white", linewidth=2.5))
plt.show()

In [ ]:
x_var = 'Line'
groupby_var = 'Division'
data_agg = data.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [data[x_var].values.tolist() for i, data in data_agg]

plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.rainbow(i/float(len(vals)-1)) for i in range(len(vals))]
#n, bins, patches = plt.hist(vals, data[x_var].unique().__len__(), stacked=True, density=False, color=colors[:len(vals)])
n, bins, patches = plt.hist(vals, data[x_var].unique().__len__()-1, stacked=True, density=False, color=colors[:len(vals)])
#Reduce length by one to make sure that the ticks and labels match

plt.legend({group:col for group, col in zip(np.unique(data[groupby_var]).tolist(), colors[:len(vals)])})
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.xlabel(x_var)
plt.ylabel("Frequency")
plt.ylim(0, 200)
plt.xticks(ticks=bins, labels=np.unique(data[x_var]).tolist(), rotation=90, horizontalalignment='left')
plt.show()

In [ ]:
data.replace(to_replace=['West'],
           value= ['W'], 
           inplace=True)
data.fillna(0)
x_var = 'Line'
groupby_var = 'Corner'
data_agg = data.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [data[x_var].values.tolist() for i, data in data_agg]
plt.figure(figsize=(16,9), dpi= 80)
colors = [plt.cm.rainbow(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, 
                            data[x_var].unique().__len__(), 
                            orientation='horizontal',
                            stacked=True, 
                            density=False, 
                            color=colors[:len(vals)])

unique_labels = data[groupby_var].tolist()
plt.legend(labels=np.unique(unique_labels), loc='best')
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.ylabel(x_var)
plt.xlabel("Count")
xlabels=np.unique(data[x_var]).tolist()
xlabels.append("") #Append to make sure that the ticks are mapped accurately
plt.yticks(ticks=bins, labels=xlabels, horizontalalignment='right')
plt.xticks([])
plt.show()

In [ ]:
x_var = 'Station Name'
groupby_var = 'Division'
data_agg = data.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [data[x_var].values.tolist() for i, data in data_agg]
colors = [plt.cm.nipy_spectral(i/float(len(vals)-1)) for i in range(len(vals))]
sns.set_context("paper", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":5})   

f=plt.figure(figsize=(16,48), dpi=100)

f=sns.histplot(
    data,
    y=x_var, 
    hue=groupby_var,
    multiple="stack",
#    palette="light:m_r",
    edgecolor=".3",
    linewidth=.5,
    color=colors[:len(vals)]
)

In [ ]:
x_var = 'Station Name'
groupby_var = 'Line'
data_agg = data.loc[:, [x_var, groupby_var]].groupby(groupby_var)
vals = [data[x_var].values.tolist() for i, data in data_agg]

plt.figure(figsize=(16,48), dpi=100)
colors = [plt.cm.nipy_spectral(i/float(len(vals)-1)) for i in range(len(vals))]
n, bins, patches = plt.hist(vals, 
                            data[x_var].unique().__len__(), 
                            stacked=True, 
                            density=False, orientation='horizontal', color=colors[:len(vals)])

plt.legend({group:col for group, col in zip(np.unique(data[groupby_var]).tolist(), colors[:len(vals)])}, 
           loc='upper right', 
           borderaxespad=0)
plt.title(f"Stacked Histogram of ${x_var}$ colored by ${groupby_var}$", fontsize=22)
plt.ylabel(x_var)
plt.xlabel("Frequency")
plt.xlim(0, 50)
plt.xticks([])
ylabels=np.unique(data[x_var]).tolist()
ylabels.append("") #Append to make sure that the ticks are mapped accurately
plt.yticks(ticks=bins, labels=ylabels, horizontalalignment='right',fontsize=7)
plt.show()

In [ ]:
temp_df = data.groupby(['Line'])['Entrance Type'].value_counts(normalize=True).mul(100).reset_index(name='Percentage')
plt.figure(figsize=(16,24), dpi=40)
g = sns.histplot(y='Line', 
                 hue='Entrance Type',
                 weights='Percentage',
                 multiple='stack',
                 data=temp_df,
                 shrink=0.99,)
plt.show()
temp_df = data.groupby(['Division'])['Entrance Type'].value_counts(normalize=True).mul(100).reset_index(name='Percentage')
plt.figure(figsize=(8,12), dpi=40)
g2 = sns.histplot(y='Division', 
                 hue='Entrance Type',
                 weights='Percentage',
                 multiple='stack',
                 data=temp_df,
                 shrink=0.99,)
plt.show()

### Which station has more routes?

In [ ]:
max_val = 0
max_line_station = []
for station in enumerate(data['Station Name'].unique()):
  temp_df = data[data["Station Name"] == station[1]]
  if len(temp_df.Route1.unique()) >= max_val:
    max_val = len(temp_df.Route1.unique())

for station in enumerate(data['Station Name'].unique()):
  temp_df = data[data["Station Name"] == station[1]]
  if len(temp_df.Route1.unique()) == max_val:
    max_line_station.append(station)

print(max_line_station)
print(max_val)

### Thanks to the https://towardsdatascience.com/exploring-and-visualizing-chicago-transit-data-using-pandas-and-bokeh-part-ii-intro-to-bokeh-5dca6c5ced10 functions on latlong to mercator

In [ ]:
import math
from ast import literal_eval
def latlong_to_mercator(latlongs):
    Coordinates = literal_eval(latlongs) 
    lat = Coordinates[0]
    lon = Coordinates[1]
    r_major = 6378137.000 #radius of the earth; default = 6378137m
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

In [ ]:
data['merc_x'] = data['Station Location'].apply(lambda x: latlong_to_mercator(x)[0])
data['merc_y'] = data['Station Location'].apply(lambda x: latlong_to_mercator(x)[1])

In [ ]:
data.T.head(35)

In [ ]:
data['freq'] = data.groupby('Station Name')['Station Name'].transform('count')

In [ ]:
data.T.head(35)
#data_g.describe()

In [ ]:
from bokeh.plotting import figure, show, output_notebook
#from bokeh.tile_providers import CARTODBPOSITRON_RETINA
from bokeh.tile_providers import get_provider, Vendors
import bokeh

tile_provider = get_provider(Vendors.CARTODBPOSITRON_RETINA)
p = figure(x_axis_type="mercator", y_axis_type="mercator")
p.add_tile(tile_provider)
p.plot_height=800
p.plot_width=1000
p.circle(x=data['merc_x'],
         y=data['merc_y'], 
         size=data['freq'],
         line_color="#555555", 
         fill_color="#CCDDEE",
         fill_alpha=0.15)
    
output_notebook()
show(p)

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.tile_providers import get_provider, Vendors
import bokeh

tile_providers = [Vendors.CARTODBPOSITRON, 
                  Vendors.CARTODBPOSITRON_RETINA, 
                  Vendors.STAMEN_TERRAIN, 
                  Vendors.STAMEN_TERRAIN_RETINA, 
                  Vendors.STAMEN_TONER, 
                  Vendors.STAMEN_TONER_BACKGROUND,
                  Vendors.STAMEN_TONER_LABELS, 
                  Vendors.OSM, 
                  Vendors.WIKIMEDIA, 
                  Vendors.ESRI_IMAGERY]

source = ColumnDataSource(data=dict(
                        x=list(data['merc_x']), 
                        y=list(data['merc_y']),
                        lines=list(data['Line']),
                        division=list(data['Division']),
                        entrance=list(data['Entrance Type']),
                        corner=list(data['Corner']),
                        vending=list(data['Vending']),
                        sizes=list(data['freq']),
                        stationname=list(data['Station Name'])))
hover = HoverTool(tooltips=[
    ("Station", "@stationname"),
    ("Division", "@division"),
    ("Line","@lines"),
    ("Entrance","@entrance"),
    ("Corner","@corner"),
    ("Vending Machines","@vending"),
])

for tp in enumerate(tile_providers):
    
    print(tp)
    tile_provider = get_provider(tp[1])
    print(tile_provider)
    p = figure(x_axis_type="mercator", 
               y_axis_type="mercator",
               tools=[hover, 'wheel_zoom','save'])
    p.add_tile(tile_provider)
    p.plot_height=1200
    p.plot_width=1400
    p.circle(x='x',
             y='y', 
             source=source,
             size='sizes',
             line_color="#2200FF", 
             fill_color="#11FF22",
             fill_alpha=0.05)
    output_notebook()
    show(p)



In [ ]:
import pandas_profiling as pdp
report = pdp.ProfileReport(data, title='Pandas Profiling Report for the dataset')
report 